In [18]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

train_preds = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/TrainingLong/TrainingPredictionsLong.csv")
train_std = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/TrainingLong/TrainingStdLong.csv")

val_preds = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/ValidationLong/ValidationPredictionsLong.csv")
val_std = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/ValidationLong/ValidationStdLong.csv")

test_preds = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/TestingLong/TestingPredictionsLong.csv")
test_std = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/TestingLong/TestingStdLong.csv")

train_actual = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/TrainingLong/TrainingActual_Long.csv")
val_actual = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/ValidationLong/ValidationActual_Long.csv")
test_actual = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/TestingLong/TestingActual_Long.csv")

# Define a function to calculate the encapsulation percentage
def calculate_encapsulation_percentage(confidence=0.7, data_name ="Training", data_setting = "Long"):
    
    prediction_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Predictions{}.csv".format(data_name, data_setting, data_name, data_setting)
    actual_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Actual_{}.csv".format(data_name,data_setting,data_name,data_setting)
    std_dev_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Std{}.csv".format(data_name, data_setting, data_name, data_setting)
    
    prediction_df = pd.read_csv(prediction_df)
    actual_df = pd.read_csv(actual_df)
    std_dev_df = pd.read_csv(std_dev_df)
    
    # Calculate Z-score for the given confidence level
    from scipy.stats import norm
    Z = norm.ppf((1 + confidence) / 2)

    # Ensure numeric conversion to handle non-numeric data gracefully
    actual_df = actual_df.apply(pd.to_numeric, errors='coerce')
    prediction_df = prediction_df.apply(pd.to_numeric, errors='coerce')
    std_dev_df = std_dev_df.apply(pd.to_numeric, errors='coerce')

    # Compute the confidence interval bounds
    lower_bounds = prediction_df - Z * std_dev_df
    upper_bounds = prediction_df + Z * std_dev_df

    # Check if actual values fall within the bounds
    encapsulation = (actual_df >= lower_bounds) & (actual_df <= upper_bounds)

    # Calculate the percentage of encapsulated values
    encapsulation_percentage = encapsulation.sum().sum() / encapsulation.size * 100

    return encapsulation_percentage

# Test the function with the loaded data
encapsulation_result = calculate_encapsulation_percentage(data_setting = "Short",data_name = "Training",confidence = 0.95)
encapsulation_result


99.744003609983